In [ ]:
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import plotly.express as px

from src.io import (
    get_techtide_hf,
    get_techtide_ionosondes,
    get_gfz_f107,
    get_gfz_hp30,
)
from src.preprocess import (
    resample_time_series,
    get_categories,
    get_solar_position,
)

In [ ]:
# START = '2024-10-14 00:00:00'
# STOP = '2024-10-14 23:59:00'

STOP_UTC_NOW = datetime.utcnow()
START_UTC = STOP_UTC_NOW - timedelta(hours=6)

STOP_UTC_NOW = STOP_UTC_NOW.strftime("%Y-%m-%d %H:%M:%S")
START_UTC = START_UTC.strftime("%Y-%m-%d %H:%M:%S")

# TechTIDE

#### HF-INT + 2-h moving average

In [ ]:
df_hf = get_techtide_hf(start=START_UTC, stop=STOP_UTC_NOW)

df_hf_30 = resample_time_series(df_hf, aggregation_function='mean').round(2)

In [ ]:
period = 2 * 2

df_hf_30[f'hf_mav_{period/2:.0f}h'] = df_hf_30['hf'].rolling(
    window=int(period)
).mean().round(2)

#### Ionosondes

In [ ]:
df_iono = get_techtide_ionosondes(
    START_UTC,
    STOP_UTC_NOW,
    iono_list=['AT138', 'FF051', 'JR055', 'PQ052', 'RO041', 'VT139'],
)

df_iono_30 = resample_time_series(
    df_iono,
    aggregation_function='median',
).round(2)

# Solar Zenith Angle (pvlib)

In [ ]:
get_solar_position(
    df_hf_30.index[-1], columns='zenith', altitude=0,
).round(1)

# GFZ

#### Hp-30

In [ ]:
df_hp_30 = get_gfz_hp30(start='2024-10-15 23:00:00', stop=STOP_UTC_NOW)

#### F10.7 (adj)

In [ ]:
get_gfz_f107()

# L1 data - TODO

In [ ]:
df_l1 = read_time_series(
    Path(DATA_IN, 'SolarWind_Projected_Merged.csv'),
    column_names=['datetime','bz','vx','rho'],
    usecols=[0,6,8,11],
).loc[START_DATE:END_DATE]

In [ ]:
df_l1_30 = resample_time_series(df_l1, aggregation_function='median')

# Newell (coupling) - TODO

In [ ]:
df_newell = read_time_series(
    Path(DATA_IN, 'newell.csv'),
    column_names=['datetime','newell'],
).loc[START_DATE:END_DATE]

In [ ]:
df_newell_30 = resample_time_series(df_newell, aggregation_function='median')

# SMR (SuperMAG partial ring current index) - TODO

In [ ]:
df_smr = read_time_series(
    Path(DATA_IN, 'SMR.csv'),
    column_names=['datetime','smr'],
).loc[START_DATE:END_DATE]

In [ ]:
df_smr_30 = resample_time_series(df_smr, aggregation_function='median')

# Auroral Electrojet - TODO

In [ ]:
df_ejet = read_time_series(
    Path(DATA_IN, 'ImageIDX.csv'),
    column_names=['il','iu','ie','datetime'],
).loc[START_DATE:END_DATE]

In [ ]:
df_ejet_30 = resample_time_series(df_ejet, aggregation_function='median')
df_ejet_30['ie'] = df_ejet_30['ie'].round(1)

We detected **anomalous IE values**

The reported values appear to be affected by instrumental **offsets** that have a time window of 1 or more consecutive days

We removed the offset by subtracting a value equal to the average of the jumps before and after the affected day(s) (```ill_dates```)

In [ ]:
df_ejet_30['ie_diff'] = df_ejet_30['ie'].diff()

ill_dates = [
    ['2015-05-31', '2015-06-01'],
    ['2015-11-27', '2015-12-01'],
    ['2015-12-04', '2015-12-07'],
    ['2018-08-29', '2018-08-31'],
    ['2018-09-01', '2018-09-08'],
    ['2019-12-27', '2020-01-01'],
]

df_ejet_30['ie_fix'] = df_ejet_30['ie']
for range_ in ill_dates:
    offset_ = np.round(
        (
            df_ejet_30.loc[range_[0], 'ie_diff'].values[0] -
            df_ejet_30.loc[range_[-1], 'ie_diff'].values[0]
        ) / 2,
        1
    )
    
    df_ejet_30.loc[
        range_[0]: pd.to_datetime(range_[1]) - pd.Timedelta(minutes=30),
        'ie_fix',
    ] -= offset_
    
df_ejet_30['ie_fix'] = df_ejet_30['ie_fix'].clip(lower=0)
df_ejet_30 = df_ejet_30.drop(columns='ie_diff')

Similarly for IU and IL

In [ ]:
df_ejet_30['iu_diff'] = df_ejet_30['iu'].diff()

ill_dates = [
    ['2015-05-31', '2015-06-01'],
    ['2015-11-27', '2015-12-01'],
    ['2015-12-04', '2015-12-07'],
    ['2019-12-27', '2020-01-01'],
]

df_ejet_30['iu_fix'] = df_ejet_30['iu']
for range_ in ill_dates:
    offset_ = np.round(
        (
            df_ejet_30.loc[range_[0], 'iu_diff'].values[0] -
            df_ejet_30.loc[range_[-1], 'iu_diff'].values[0]
        ) / 2,
        1
    )
    
    df_ejet_30.loc[
        range_[0]: pd.to_datetime(range_[1]) - pd.Timedelta(minutes=30),
        'iu_fix',
    ] -= offset_
    
df_ejet_30 = df_ejet_30.drop(columns='iu_diff')

In [ ]:
df_ejet_30['il_diff'] = df_ejet_30['il'].diff()

ill_dates = [
    ['2018-08-29', '2018-08-31'],
    ['2018-09-01', '2018-09-08'],
]

df_ejet_30['il_fix'] = df_ejet_30['il']
for range_ in ill_dates:
    offset_ = np.round(
        (
            df_ejet_30.loc[range_[0], 'il_diff'].values[0] -
            df_ejet_30.loc[range_[-1], 'il_diff'].values[0]
        ) / 2,
        1
    )
    
    df_ejet_30.loc[
        range_[0]: pd.to_datetime(range_[1]) - pd.Timedelta(minutes=30),
        'il_fix',
    ] -= offset_
    
df_ejet_30 = df_ejet_30.drop(columns='il_diff')

Finally, we discretise IE, IL and IU in categories according to their variation

In [ ]:
hours = 6
time_steps = 2 * hours

In [ ]:
_, labels = get_categories(
    df_ejet_30['ie_fix'],
    window=time_steps,
    zero_phase=False,
)

df_ejet_30['ie_variation'] = np.insert(labels, 0, 0, axis=0)

In [ ]:
_, labels = get_categories(
    df_ejet_30['iu_fix'],
    window=time_steps,
    zero_phase=False,
)

df_ejet_30['iu_variation'] = np.insert(labels, 0, 0, axis=0)

In [ ]:
_, labels = get_categories(
    df_ejet_30['il_fix'],
    window=time_steps,
    zero_phase=False,
)

df_ejet_30['il_variation'] = np.insert(labels, 0, 0, axis=0)

Here we construct IE, IL and IU moving averages with several rolling windows (3, 6, 12 and 24 hours)

In [ ]:
# n. of periods needed to get 3, 6, 12, 24 hours
periods = [2 * per_ for per_ in [3, 6, 12, 24]]

for per_ in periods:
    # IE moving average
    df_ejet_30[f'ie_mav_{per_/2:.0f}h'] = df_ejet_30['ie_fix'].rolling(
        window=int(per_)
    ).mean().round(1)
    
    # IU moving average
    df_ejet_30[f'iu_mav_{per_/2:.0f}h'] = df_ejet_30['iu_fix'].rolling(
        window=int(per_)
    ).mean().round(1)
    
    # IL moving average
    df_ejet_30[f'il_mav_{per_/2:.0f}h'] = df_ejet_30['il_fix'].rolling(
        window=int(per_)
    ).mean().round(1)
    
df_ejet_30 = df_ejet_30.drop(columns=['ie','il','iu']).dropna()

## Dataset creation

In [ ]:
df_j = df_ejet_30.merge(
    df_tid_30_['quality_index'],
    how='left',
    left_index=True,
    right_index=True,
).merge(
    df_hf_30,
    how='left',
    left_index=True,
    right_index=True,
).merge(
    df_solar['f_107_adj'],
    how='left',
    left_index=True,
    right_index=True,
).merge(
    df_hp_30,
    left_index=True,
    right_index=True,
).merge(
    df_smr_30,
    left_index=True,
    right_index=True,
).merge(
    df_l1_30,
    left_index=True,
    right_index=True,
).merge(
    df_newell_30,
    left_index=True,
    right_index=True,
).merge(
    df_ionosondes_30,
    left_index=True,
    right_index=True,
)

# Solar data need to be repeated, since they're provided on a daily basis only
df_j['f_107_adj'] = df_j['f_107_adj'].ffill()

# Solar zenith angle
df_j['solar_zenith_angle'] = get_solar_position(
    df_j.index, columns='zenith', altitude=0,
).round(1)

In [ ]:
assert df_j.reset_index().duplicated('datetime').sum() == 0

Construct the actual **target**, a boolean column which is set to 1 whenever a **TID event** is reported **within a 3-hours timeframe**

In [ ]:
steps = 2 * FORECAST_HOURS_IN_ADVANCE

df_j[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h'] = df_j['quality_index'].rolling(
    window=steps+1, min_periods=1
).sum().gt(0).shift(
    -steps, fill_value=False
).replace(
    {True: 1, False: 0}
)

In [ ]:
# df_j['solar_zenith_angle'].apply(np.real).describe()

## Dump

In [ ]:
df_j.drop(
    columns=['quality_index'],
).to_pickle(
    Path(DATA_OUT, 'df_dataset.pickle')
)